# Pupular Data Science Questions

For this project, we are working for a company that publishes online content and wants to start writing about about data science. They want to start a new blog or a social media account and they have asked me to help them figure out what topics they should we write about.

To answer this question we have decided to look into some data to figure what are the most populat topics in datascience.

For this we will analyze the posts in Data Science Stack Exchange's website and find out what are the most popular topics that have been published and read.



## Exploring Data Science Stack Exchange's database 
Stack Exchange Data Explorer (SEDE)
https://data.stackexchange.com/datascience/query/new 

We use "Viewing query" to explore the data and select the columns that we want  to use for our project.



Then we download the data as .csv to use.


In [1]:
"""
SELECT Id, PostTypeId, CreationDate, Score, ViewCount, Tags,
        AnswerCount, FavoriteCount
FROM Posts
WHERE CreationDate < '2020-01-01' 
      AND Posts.CreationDate > '2018-12-31';
"""

"\nSELECT Id, PostTypeId, CreationDate, Score, ViewCount, Tags,\n        AnswerCount, FavoriteCount\nFROM Posts\nWHERE CreationDate < '2020-01-01' \n      AND Posts.CreationDate > '2018-12-31';\n"

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

%matplotlib inline

In [3]:


questions = pd.read_csv("2019_questions.csv", parse_dates=['CreationDate'])
print(questions.head())

FileNotFoundError: [Errno 2] No such file or directory: '2019_questions.csv'

In [ ]:
questions.info()


## Cleaning data


In [ ]:
# Count each value in FavouriteCount:
print(questions['FavoriteCount'].value_counts(dropna=False))

#Find null values in FavouriteCount. Not all questions are sobemody's favourite
nan = questions['FavoriteCount'].isna().sum()

print("nan values:")
print(+nan)


There are many posts that have between 1 and 4 favourite votes, and only a few that have between 5 and 16 votes. 
Since there are many null values, we could transform them to 0 values, since it is the same in this case. 

In [ ]:
#Fill missing values in favourite count with 0
questions['FavoriteCount'] = questions['FavoriteCount'].fillna(0).astype(int)

#Check for missing values and confirm operation
print("nan values:")
print(questions['FavoriteCount'].isna().sum())

print("Value counts:")
print(questions['FavoriteCount'].value_counts(dropna=False))

In [ ]:
#Convert creation date to datetime object
questions['CreationDate'] = pd.to_datetime(questions['CreationDate'])


In [ ]:
#Check data 
questions.info()


In [ ]:
#Clean de Tags column 
questions['Tags'] = questions['Tags'].str.replace('><', ",").str.replace('<', '').str.replace('>', '')
print(questions['Tags'].head(5))


In [ ]:
# Transform the values in the tag columns to a list of lists
questions['Tags'] = questions['Tags'].str.split(',')
print(questions['Tags'].head())

## Most Used and Most Viewed tags

Now that we cleaned the data, we need to determine the most popular tags based on two different popularity proxies: 
- how many times the tag was used
- and how many times a question with that tag was viewed.

In [ ]:
# Count how many times each tag was used.
tag_count = dict()
for tag_list in questions['Tags']:
    for tag in tag_list:
        if tag in tag_count:
            tag_count[tag] += 1
        else:
            tag_count[tag] = 1
            
# convert the dictionary into a dataframe
tag_count_df = pd.DataFrame(list(tag_count.items()), columns = ['tag_name','tag_count'])

# get top 10 most popular tags
top_10_used = tag_count_df.sort_values("tag_count", ascending=False).head(10)

top_10_used

In [ ]:
# Count how many times each tag was viewed

tag_views = dict()

for index, row in questions.iterrows():
    # loop through the list of each row
    for tag in row['Tags']:
        # check if the tag is in the dictionary and add the values
        if tag in tag_views:
            tag_views[tag] += row['ViewCount']
        else:
            tag_views[tag] = row['ViewCount']

# convert the dictionary into a dataframe
tag_views_df =  pd.DataFrame(list(tag_views.items()), columns = ['tag_name','view_count'])

# get top 10 most viewed tags
top_10_viewed = tag_views_df.sort_values("view_count", ascending=False).head(10)

top_10_viewed

In [ ]:
# Visualizations for the top tags of each of the above results.

# Bar plot for the top 10 most popular tags
top_10_used.plot(kind = 'barh', x = 'tag_name', y = 'tag_count', legend = False, title = 'Top 10 Most used Tags')
plt.show()

In [ ]:
# Bar plot for the top 10 most viewed tags

top_10_viewed.plot(kind = 'barh', x = 'tag_name', y = 'view_count', legend = False, title = 'Top 10 Most viewed Tags')
plt.show()

## Relations Between Tags



In [ ]:
# Create a dataframe with the top 20 most used and most viewed tags
most_viewed = tag_views_df.sort_values("view_count", ascending=False).head(20)
most_used = tag_count_df.sort_values("tag_count", ascending=False).head(20)

# set tag_name as index
most_viewed.set_index("tag_name", inplace=True)
most_used.set_index("tag_name", inplace=True)

In [ ]:
# Find which tags are in most_used, but not in most_viewed
in_used = pd.merge(most_used, most_viewed, how="left", left_index=True, right_index=True)
in_used

In [ ]:
# Find which tags are in most_viewed, but not in most_used
in_viewed = pd.merge(most_used, most_viewed, how="right", left_index=True, right_index=True)
in_viewed

The tags present in most_used and not present in most_viewed are:
- machine-learning-model
- statistics
- predictive-modeling
- r

And the tags present in most_viewed but not in most_used are:
- csv
- pytorch
- dataframe

Some tags also stand out as being related. For example, python is related to pandas, as we can find both pythons and pandas in the same country — or better yet, because pandas is a Python library. So by writing about pandas, we can actually simultaneously tackle two tags.

## Is deep learning just a fad?

After taking a look at the most used and viewed tags, we start to wonder what relationship is there between the most popular tags.
To answer this question, we will analyze all the questions from the SEDE (not just from 2019) to track the interest in deep learning across time. 

We will:
- Count how many deep learning questions are asked per time period.
- The total amount of questions per time period.
- How many deep learning questions there are relative to the total amount of   questions per time period.

In [ ]:
"""
SELECT Id, CreationDate, Tags
  FROM posts
 WHERE PostTypeId = 1;
 """

In [ ]:
# Read the file all_questions.csv into a dataframe.
all_questions = pd.read_csv("all_questions.csv", parse_dates=["CreationDate"])

all_questions.head()

In [ ]:
# clean the Tags column by Removing brackets from Tags columns
all_questions['Tags'] = all_questions['Tags'].str.replace('><', ',').str.replace('<', '').str.replace('>', '')

# split the Tags column into list
all_questions['Tags'] = all_questions['Tags'].str.split(',')

In [ ]:
all_questions.head()

In [ ]:
 # get the latest date in the list
all_questions['CreationDate'].max()

In [ ]:
 # get the oldest date in the list
all_questions['CreationDate'].min()

Based on the previous part of the project, we will take some of the most popular tags that we believe are associated with deep learning to see how many times these tags show up in questions. This will give us a definition of deep learning that we can use to analyze the use of tags in specific time periods.


In [ ]:
# Create a list of tags relating to deep-learning topics
deep_learning = ["python",
             "machine-learning",
             "keras",
             "deep-learning",
             "neural-network",
             "scikit-learn",
             "tensorflow",
             "classification"
            ]

# Define a function to check for matches to deep_learning
def deep_tags(row):
    tags = ','.join(row['Tags'])
    for tag in deep_learning:
        if tag in tags:
            return "deep-learning"

    return "Other"

# Apply the  function to create a new column
all_questions['Question_Category'] = all_questions.apply(deep_tags, axis=1)

In [ ]:
all_questions.head(10)

In [ ]:
# Using the month timeframe to track interest in deep learning
# How many times in a month were tags related to deep learning used ?
all_questions['Creation_Month'] = all_questions['CreationDate'].dt.strftime('%Y%m')
all_questions['Creation_Month'] = pd.to_datetime(all_questions['Creation_Month'], format='%Y%m')

In [ ]:
# Count how many deep learning questions are asked all together.
all_questions['Question_Category'].value_counts()

In [ ]:
# Count how many deep learning questions are asked in a month.
grouped = all_questions.groupby(['Creation_Month', 'Question_Category']).size().reset_index(name='Count')

# pivot the table to have columns for each question type
pv_table = grouped.pivot(index='Creation_Month', columns='Question_Category', values='Count')

In [ ]:
pv_table.head()

In [ ]:
# plot deep learning questions count across each month
ax = pv_table.plot(kind='line', figsize=(16,8))
ax.set_xlabel('Date')
ax.set_ylabel('Number of Questions')
ax.set_title('Question Category')

# set x ticks for every other month
x_ticks = pv_table.index[::2]
ax.set_xticks(x_ticks)

plt.show()

The data clearly shows that people have been asking questions about topics related to deep learning for several years and it seems to be the most adequate  subject to write about. 